In [173]:
import rpy2
import rpy2.robjects as robjects
r = robjects.r
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import statsmodels.api as sm
import pandas as pd
from itertools import combinations
import numpy as np

# Download Carlson Data

In [174]:
base = importr('base')
utils = importr('utils')
r('install.packages("FindIt", repos="https://CRAN.R-project.org/")')
findit = importr('FindIt')
'''using data from example in FindIt'''
robjects.r('data("Carlson", package = "FindIt")')
carlson = robjects.r['Carlson']
carlson_df = pandas2ri.rpy2py(carlson)
print(carlson_df)

'''trying a couple variables to use the 2 way amie'''
new_Record = robjects.r('''
Carlson$newRecordF<- factor(Carlson$newRecordF,ordered=TRUE,
levels=c("YesLC", "YesDis","YesMP",
"noLC","noDis","noMP","noBusi"))''')
promise = robjects.r('''
Carlson$promise <- factor(Carlson$promise,ordered=TRUE,levels=c("jobs","clinic","education"))
''')
model = robjects.r('''

fit2 <- CausalANOVA(formula=won ~ newRecordF + promise,
int2.formula = ~ newRecordF:promise,
data=Carlson, pair.id=Carlson$contestresp,diff=TRUE,
cluster=Carlson$respcodeS, nway=2)
''')

R[write to console]: trying URL 'https://CRAN.R-project.org/bin/macosx/big-sur-arm64/contrib/4.3/FindIt_1.2.0.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 430138 bytes (420 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console


The downloaded binary packages are in
	/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T//RtmpwUf5No/downloaded_packages
      won  newRecordF  promise  coeth_voting  relevantdegree  respcodeS   
1       1           6        1             2               1          1  \
2       0           7        3             1               2          1   
3       1           7        3             1               2          1   
4       0           3        2             1               2          1   
5       0           7        1             1               1          1   
...   ...         ...      ...           ...             ...        ...   
3228    1           4        1             1               2        544   
3229    1           5        1             2               2        544   
3230    0           2        2             1               1        544   
3231    0           7        1             2               2        544   
3232    1           3        1             1          

In [100]:
#One Hot Encode
enc = OneHotEncoder(handle_unknown='ignore')
X = carlson_df.iloc[:,1:-2]
y = carlson_df.iloc[:,0].to_numpy() 
trans_x = enc.fit_transform(X).toarray() 
df =pd.DataFrame(trans_x)
print(df)
#0-6 are record, 7-9 are promise, 10-11 are coeth, 12-13 is degree
record_list = [i for i in range(7)]
promise_list = [7,8,9]
coeth_list = [10,11]
degree_list = [12,13]
for record in record_list:
    for promise in promise_list:
        df[f"{record}*{promise}"] = df[record] * df[promise]
        for coeth in coeth_list:
            df[f"{record}*{promise}*{coeth}"] = df[record] * df[promise] * df[coeth]
            for degree in degree_list:
                df[f"{record}*{promise}*{coeth}*{degree}"] = df[record] * df[promise] * df[degree] * df[coeth]
        for degree in degree_list:
                df[f"{record}*{promise}*{degree}"] = df[record] * df[promise] * df[degree]
    for coeth in coeth_list:
        df[f"{record}*{coeth}"] = df[record] * df[coeth]
        for degree in degree_list:
             df[f"{record}*{coeth}*{degree}"] = df[record] * df[coeth] * df[degree]
    for degree in degree_list:
        df[f"{record}*{degree}"] = df[record] * df[degree]

print(len(df.columns))

       0    1    2    3    4    5    6    7    8    9    10   11   12   13
0     0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0
1     0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0
2     0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0
3     0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0
4     0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
3227  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0
3228  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0
3229  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0
3230  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0
3231  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0

[3232 rows x 14 columns]
259


/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T/ipykernel_61654/3295271560.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{record}*{coeth}*{degree}"] = df[record] * df[coeth] * df[degree]
/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T/ipykernel_61654/3295271560.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{record}*{coeth}"] = df[record] * df[coeth]
/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T/ipykernel_61654/3295271560.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [101]:
# train test split
column_names = ['bias']
for column in df.columns:
    column_names.append(column)

T = df.to_numpy()
T = np.insert(T, 0, 1, axis=1)
T_train, T_test, y_train, y_test = train_test_split(T, y, test_size=0.1)
print(T_train)
print(y_train)

[[1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]]
[1 1 0 ... 1 0 1]


In [109]:


model2 = Lasso(alpha=0.002)
model2.fit(T_train, y_train)
y_pred = model2.predict(T_test)
# Retrieve the non-zero values using the indices
nonzero_indices = np.nonzero(model2.coef_)
print(nonzero_indices[0])
nonzero_values = model2.coef_[nonzero_indices]
count = y_test.shape[0]
correct = 0
for pred, act in zip(y_pred,y_test):
    if (pred >=0.5 and act ==1) or (pred <0.5 and act ==0):
        correct+=1
print(correct/count)

[  1   7  10  11  12  26  33  41  45  60  80  95 102 114 115 118 138 146
 150 157 181 187 191 208 225 258]
0.5370370370370371


In [110]:
selected_column_names = [column_names[i] for i in nonzero_indices[0]]

for name, coef in zip(selected_column_names, model2.coef_[nonzero_indices[0]]):
    if abs(coef) < 0.01:
        continue
    print(name, coef)

6 -0.04844313793737722
9 0.03426664288962077
10 -0.017474636608257394
0*8*10*12 0.07009696728032619
0*9 0.010548542961584804
0*11 0.11723916586365445
1*11 0.09075455977521789
2*8*10 -0.03168264055035738
2*8*13 -0.01668977731431804
2*10*13 -0.015548064413990982
2*11 0.08255283220049071
3*9*13 -0.056917339807152856
3*11 -0.05329258308113674
4*7*10*12 0.026921410338982276
4*11*13 0.03584250028712883
5*7*10 0.06322145321896436
6*12 -0.0272986056427621


In [75]:
print(column_names)

['bias', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '0*7', '0*7*10', '0*7*10*12', '0*7*10*13', '0*7*11', '0*7*11*12', '0*7*11*13', '0*7*12', '0*7*13', '0*8', '0*8*10', '0*8*10*12', '0*8*10*13', '0*8*11', '0*8*11*12', '0*8*11*13', '0*8*12', '0*8*13', '0*9', '0*9*10', '0*9*10*12', '0*9*10*13', '0*9*11', '0*9*11*12', '0*9*11*13', '0*9*12', '0*9*13', '0*10', '0*10*12', '0*10*13', '0*11', '0*11*12', '0*11*13', '0*12', '0*13', '1*7', '1*7*10', '1*7*10*12', '1*7*10*13', '1*7*11', '1*7*11*12', '1*7*11*13', '1*7*12', '1*7*13', '1*8', '1*8*10', '1*8*10*12', '1*8*10*13', '1*8*11', '1*8*11*12', '1*8*11*13', '1*8*12', '1*8*13', '1*9', '1*9*10', '1*9*10*12', '1*9*10*13', '1*9*11', '1*9*11*12', '1*9*11*13', '1*9*12', '1*9*13', '1*10', '1*10*12', '1*10*13', '1*11', '1*11*12', '1*11*13', '1*12', '1*13', '2*7', '2*7*10', '2*7*10*12', '2*7*10*13', '2*7*11', '2*7*11*12', '2*7*11*13', '2*7*12', '2*7*13', '2*8', '2*8*10', '2*8*10*12', '2*8*10*13', '2*8*11', '2*8*11*12', '2*8*1

# Try grouping levels

In [175]:
#group levels
print(carlson_df)
carlson_df['newRecordF'] = np.where(carlson_df['newRecordF'] <= 3, -1, 1)
print(carlson_df)
carlson_df['promise'] = np.where(carlson_df['promise'] <= 2, -1, 1)
carlson_df['relevantdegree'] = np.where(carlson_df['relevantdegree'] <= 1, -1, 1)
carlson_df['coeth_voting'] = np.where(carlson_df['coeth_voting'] <= 1, -1, 1)
X = carlson_df.iloc[:,1:-2]
y = carlson_df.iloc[:,0].to_numpy() *2 -1

degree = 14
pf = preprocessing.PolynomialFeatures(
    degree=degree, interaction_only=True, include_bias=True,
)
T = pf.fit_transform(X)
print((T[:,1]))

      won  newRecordF  promise  coeth_voting  relevantdegree  respcodeS   
1       1           6        1             2               1          1  \
2       0           7        3             1               2          1   
3       1           7        3             1               2          1   
4       0           3        2             1               2          1   
5       0           7        1             1               1          1   
...   ...         ...      ...           ...             ...        ...   
3228    1           4        1             1               2        544   
3229    1           5        1             2               2        544   
3230    0           2        2             1               1        544   
3231    0           7        1             2               2        544   
3232    1           3        1             1               1        544   

       contestresp  
1     1.014310e+09  
2     1.014310e+09  
3     1.014210e+09  
4     1.014210e

In [186]:
# train test split
T_train, T_test, y_train, y_test = train_test_split(T, y, test_size=0.1)
print(T_train)
print(y_train)

model2 = Lasso(alpha=0.03)
model2.fit(T_train, y_train)
y_pred = model2.predict(T_test)
# Retrieve the non-zero values using the indices
nonzero_indices = np.nonzero(model2.coef_)
print(nonzero_indices[0])
nonzero_values = model2.coef_[nonzero_indices]
count = y_test.shape[0]
correct = 0
for pred, act in zip(y_pred,y_test):
    if (pred >=0 and act ==1) or (pred <0 and act ==-1):
        correct+=1
print(correct/count)

[[ 1.  1. -1. ...  1. -1. -1.]
 [ 1.  1. -1. ... -1.  1.  1.]
 [ 1.  1. -1. ... -1.  1.  1.]
 ...
 [ 1.  1. -1. ... -1.  1.  1.]
 [ 1.  1. -1. ...  1. -1. -1.]
 [ 1. -1. -1. ... -1. -1.  1.]]
[-1  1 -1 ... -1  1 -1]
[ 1  3  6 11]
0.5648148148148148


In [187]:

for name, coef in zip(pf.get_feature_names_out()[nonzero_indices[0]], model2.coef_[nonzero_indices[0]]):
    print(name, coef)

newRecordF -0.06328695414373924
coeth_voting 0.04020098171842715
newRecordF coeth_voting -0.03893670967246093
newRecordF promise coeth_voting 0.015873425958958208
